# Optimiser un programme Python (et C) en utilisant des instructions SIMD

![SIMD example](SIMD_cpu_diagram1.svg "Wikipédia https://en.wikipedia.org/wiki/SIMD#/media/File:SIMD_cpu_diagram1.svg") 

[SIMD](https://en.wikipedia.org/wiki/SIMD) signifie **Single Instruction Multiple Data**

Ces instructions permettent d'exécuter la même opération sur plusieurs données simultanément, améliorant ainsi la rapidité d'exécution de votre programme.

Si vous faîtes des additions quatre par quatre, vous irez quatre fois plus vite que si vous les faîtes une par une !  
C'est l'idée sous-jacente de ces instructions, dîtes vectorielles.

Ce TP est inspiré d'une série d'articles de la revue informatique [MISC N°70 - HPC, Big Data](https://boutique.ed-diamond.com/numeros-deja-parus/506-misc70.html).  

Nous vous proposons ici un petit programme en langage `C` très simple qui additionne le contenu de 2 tableaux de réels.

Nous allons le compiler normalement, puis de nouveau après l'avoir transformé en utilisant des instructions SIMD ajoutées manuellement. Enfin nous demanderons au compilateur `gcc` de le faire pour nous.

Pour finir, nous réaliserons les mêmes optimisations en Python grâce à la librairie [Numba](https://numba.pydata.org)


## Compilation sans optimisation SIMD

Voici un petit programme C que nous allons compiler sans optimisation SIMD.

In [1]:
%%file compute_simple.c
#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>

double get_timestamp()
{
    struct timeval tv;
    gettimeofday(&tv, NULL);
    return (double)tv.tv_sec + ((double)tv.tv_usec)/1000000.0;
}

void compute(float* r, float const* a, float const* b, const size_t n)
{
    for (size_t i = 0; i < n; i++) {
        r[i] = a[i] + b[i] ;
    }
}


int main(int argc, char** argv)
{
    double start_t, end_t;
    double diff_t;
    int dummy;

    float *data1, *data2, *result;

    if (argc < 2) {
        printf("Usage: %s <number of items in table>\n", argv[0]);
        return 1;
    }

    const size_t n = atoll(argv[1]);  /* convert second param, first one is program name, to long integer */
    printf("Using a table of %lu floats\n", (unsigned long)n);

    /* Initializing the table content */
    dummy = posix_memalign((void**) &data1, 32, sizeof(float)*n);
    dummy = posix_memalign((void**) &data2, 32, sizeof(float)*n);
    dummy = posix_memalign((void**) &result, 32, sizeof(float)*n);

    for (size_t i = 0; i < n; i++) {
        data1[i] = (float)i;  /* items in data1 contain value of their indice */
        data2[i] = (float)i*2;
    }

    /* Start time */
    start_t = get_timestamp();

    /* Computing  */
    compute(result, data1, data2, n);

    /* End time and duration */
    end_t = get_timestamp();
    diff_t = end_t - start_t;

   printf("Execution time : %f seconds\n", diff_t);
   printf("%f %f %f %f\n", result[0], result[1], result[2], result[3]);

    return 0;
}



Writing compute_simple.c


Quelques explications sur ce programme pour les personnes qui ne programment pas en langage C:
    
* La fonction compute est celle que nous souhaitons étudier
* Elle additionne 2 tableaux de réels simple précision (4 octets) et retourne le résultat dans un troisième
* Elle prend 3 adresses mémoires en entrée:
    * La première est celle du tableau de réels qui contiendra le résultat
    * Les deux autres sont les deux tableaux qu'il convient d'additionner
* Son paramètre `n` correspond au nombre d'éléments des tableaux
* La boucle `for` est simple à comprendre, elle additionne les éléments de même rang entre eux et stocke le résultat dans le tableau final
* Le programme attend un paramètre en entrée: le nombre de réels dans chaque tableau
* Les fonctions `posix_memalign` allouent la taille mémoire en octets de ces tableaux ; soit la taille d'un réel multipliée par le nombre d'éléments

Maintenant compilons avec le minimum d'optimisation ce code C.

In [2]:
!gcc compute_simple.c  -o compute_simple.bin

Enfin, nous pouvons exécuter le programme généré.  
**ATTENTION** nous générons 1 milliard de réels simple précision (4 octets) dans 3 tableaux.  
Il vous faudra 12Go de RAM pour que cela fonctionne sans swapper, sinon changez le nombre passé en ligne de commande.

In [3]:
%%time
!./compute_simple.bin 1000000000  # CAUTION: 1 Giga of floats (size 4 bytes) equal 3x4 = 12Go of memory allocated

Using a table of 1000000000 floats
Execution time : 3.041516 seconds
0.000000 3.000000 6.000000 9.000000
CPU times: user 81.6 ms, sys: 29.2 ms, total: 111 ms
Wall time: 7.4 s


Le temps d'exécution peut sembler plus long que le temps affiché, car il ne tient pas compte de l'initialisation du tableau.  
Ce qui est confirmé par le retour du mot clef `%%time`.

## Compilation en utilisant les instructions SIMD

Les instructions SIMD dépendent de l'architecture de votre matériel. Il faut se référer à la documentation du fabricant de votre microprocesseur pour savoir lesquelles utiliser.

Les instructions de type SSE et AVX sur architecture Intel ou encore NEON sur ARM en sont des exemples. 

Avec Intel, nous allons utiliser les fonctions [intrinsèques](http://software.intel.com/en-us/articles/intel-intrinsics-guide).  

Il existe plusieurs variantes d'instructions SIMD, comme SSE, SSE2, SSE3, SSSE3, SSE4.1, AVX, AVX2, AVX512 qui sont liées à la génération de votre processeur. 

Pour connaître les possibilités de votre microprocesseur, la commande `lscpu` vous indique les flags/drapaux disponibles.



```bash
$ lscpu
Architecture :                          x86_64
Mode(s) opératoire(s) des processeurs : 32-bit, 64-bit
Boutisme :                              Little Endian
Processeur(s) :                         12
...
Identifiant constructeur :              GenuineIntel
...
Drapaux :                               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 ss ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc art arch_perfmon pebs bts rep_good nopl xtopology nonstop_tsc cpuid aperfmperf tsc_known_freq pni pclmulqdq dtes64 monitor ds_cpl vmx smx est tm2 ssse3 sdbg fma cx16 xtpr pdcm pcid sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave avx f16c rdrand lahf_lm abm 3dnowprefetch cpuid_fault epb invpcid_single pti ssbd ibrs ibpb stibp tpr_shadow vnmi flexpriority ept vpid ept_ad fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm mpx rdseed adx smap clflushopt intel_pt xsaveopt xsavec xgetbv1 xsaves dtherm ida arat pln pts hwp hwp_notify hwp_act_window hwp_epp md_clear flush_l1d
```

Celui-ci supporte jusqu'à [SSE4.2](https://fr.wikipedia.org/wiki/SSE4) et [AVX2](https://fr.wikipedia.org/wiki/Advanced_Vector_Extensions).

Nous allons transformer le programme pour utiliser les instructions SSE. Ceci va compliquer un brin le code d'origine et cassera sa portabilité car il ne sera maintenant compatible qu'avec les machines disposant de ces instructions.

In [4]:
%%file compute_sse.c
#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>
#include <immintrin.h>

double get_timestamp()
{
    struct timeval tv;
    gettimeofday(&tv, NULL);
    return (double)tv.tv_sec + ((double)tv.tv_usec)/1000000.0;
}

void compute_sse(float* r, float const* a, float const* b, const int n)
{
    __m128 sse_a, sse_b, sse_r;

    size_t nb_items = (128 / (sizeof(float)*8));
    const size_t nsse = (n/nb_items)*nb_items;

    for (size_t i = 0; i < nsse; i += 4) {
        sse_a = _mm_load_ps(&a[i]);
        sse_b = _mm_load_ps(&b[i]);
        sse_r = _mm_add_ps(sse_a, sse_b);
        _mm_store_ps(&r[i], sse_r);
    }
}


int main(int argc, char** argv)
{
    double start_t, end_t;
    double diff_t;
    int dummy;

    float *data1, *data2, *result;

    if (argc < 2) {
        printf("Usage: %s <number of items in table>\n", argv[0]);
        return 1;
    }

    const size_t n = atoll(argv[1]);
    printf("Using a table of %lu floats\n", (unsigned long)n);

    /* Initializing the table content */
    dummy = posix_memalign((void**) &data1, 32, sizeof(float)*n);
    dummy = posix_memalign((void**) &data2, 32, sizeof(float)*n);
    dummy = posix_memalign((void**) &result, 32, sizeof(float)*n);

    for (size_t i = 0; i < n; i++) {
        data1[i] = (float)i;  /* items in data1 contain value of their indice */
        data2[i] = (float)i*2; /* the negative version if the indice */
    }


    /* Start time */
    start_t = get_timestamp();

    /* Computing  */
    compute_sse(result, data1, data2, n);

    /* End time and duration */
    end_t = get_timestamp();
    diff_t = end_t - start_t;

   printf("Execution time using manual sse: %f seconds\n", diff_t);
   printf("%f %f %f %f\n", result[0], result[1], result[2], result[3]);

    return 0;
}


Writing compute_sse.c


Expliquons un peu le code:

* L'instruction `#include <immintrin.h>` permet d'importer les fonctions intrinsèques d'Intel.
* La fonction `compute` a été renommée en `compute_sse`.
* Elle instancie 3 variables *sse_a*, *sse_b* et *sse_r* qui représentent un ensemble de 4 réels en simple précision (`__m128`).
* Les fonctions `_mm_load_ps` chargent 4 réels dans les 4 registres sse qui vont pouvoir calculer en utilisant ces valeurs.
* La fonction `_mm_add_ps` additionne les 2 ensembles sse de 4 réels.  
* Pour finir `_mm_store_ps` convertit le résultat des variables sse dans le tableau de réels.


La fonction `_mm_load_ps` a besoin de disposer d'une adresse mémoire qui soit multiple de 16, d'où l'utilisation de la fonction `posix_memalign` pour l'allocation mémoire.  Ici nous l'avons alignée sur un multiple de 32 pour préparer une version utilisant 8 registres de réels en simple précision: `__m256` 

Compilons le code:

In [5]:
!gcc compute_sse.c  -o compute_sse.bin

In [6]:
%%time
!./compute_sse.bin 1000000000  # CAUTION: 1 Giga of floats (size 4 bytes), so 3x4 = 12Go of memory allocated

Using a table of 1000000000 floats
Execution time using manual sse: 2.022335 seconds
0.000000 3.000000 6.000000 9.000000
CPU times: user 76.8 ms, sys: 26.3 ms, total: 103 ms
Wall time: 6.4 s


Nous passons de - *grosso modo* - 3 secondes à 2 secondes.  
Pourquoi le temps n'est-il pas divisé par 4 puisque nous faisons 4 fois moins d'itérations?  

Cela est dû, *je pense*, au temps de tranfert des données de type `float` vers les variables sse et inversement. Mais je ne suis pas un expert sur le sujet.

Pour gagner plus de temps il est possible d'utiliser des structures encore plus grandes comme `__m256` et `__m512` disponibles avec les instructions AVX/AVX2/AVX512.



### Exercice

Convertir le code pour utiliser les instructions sse sur 256 bits avec AVX.

Pour cela vous allez devoir:

* Changer la structure `__m128` par `__m256`
* Incrémenter le pas de la boucle for de 8 au lieu de 4
* Utiliser les instructions sse avec le préfixe `_mm256_` au lieu du préfixe `_mm_`  
  Exemple `_mm_load_ps` sera remplacé par `_mm256_load_ps`
* Enfin, et j'y ai perdu une journée, ajouter l'option de compilation `-mavx`
* Aligner la mémoire des tableaux sur un multiple de 32 au lieu de 16 (déjà fait)

<button data-toggle="collapse" data-target="#mm256" class='btn btn-primary'>Solution</button>
<div id="mm256" class="collapse">

```c
#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>
#include <immintrin.h>

double get_timestamp()
{
    struct timeval tv;
    gettimeofday(&tv, NULL);
    return (double)tv.tv_sec + ((double)tv.tv_usec)/1000000.0;
}

void compute_sse(float* r, float const* a, float const* b, const int n)
{
    __m256 sse_a, sse_b, sse_r;

    int fs = sizeof(float);
    int sse_mode = 256;

    size_t nb_items = (sse_mode / (fs*8));
    const size_t nsse = (n/nb_items)*nb_items;

    printf("Processing items by %d\n", (int)nb_items);

    for (size_t i = 0; i < nsse; i += nb_items) {
        sse_a = _mm256_load_ps(&a[i]);
        sse_b = _mm256_load_ps(&b[i]);
        sse_r = _mm256_add_ps(sse_a, sse_b);
        _mm256_store_ps(&r[i], sse_r);
    }
}


int main(int argc, char** argv)
{
    double start_t, end_t;
    double diff_t;
    int dummy;

    float *data1, *data2, *result;

    if (argc < 2) {
        printf("Usage: %s <number of items in table>\n", argv[0]);
        return 1;
    }

    const size_t n = atoll(argv[1]);
    printf("Using a table of %lu floats. Each float is %d bytes\n", (unsigned long)n, (int)sizeof(float));

    /* Initializing the table content */
    dummy = posix_memalign((void**) &data1, 32, sizeof(float)*n);
    dummy = posix_memalign((void**) &data2, 32, sizeof(float)*n);
    dummy = posix_memalign((void**) &result, 32, sizeof(float)*n);

    for (size_t i = 0; i < n; i++) {
        data1[i] = (float)i;  /* items in data1 contain value of their indice */
        data2[i] = (float)i*2; /* the negative version if the indice */
    }


    /* Start time */
    start_t = get_timestamp();

    /* Computing  */
    compute_sse(result, data1, data2, n);

    /* End time and duration */
    end_t = get_timestamp();
    diff_t = end_t - start_t;

   printf("Execution time using manual sse: %f seconds\n", diff_t);
   printf("%f %f %f %f\n", result[0], result[1], result[2], result[3]);

    return 0;
}


```

```bash
$ gcc compute_sse_256.c -mavx -o compute_sse_256.bin
$ ./compute_sse_256.bin 1000000000
Using a table of 1000000000 floats. Each float is 4 bytes
Processing items by 8
Execution time using manual sse: 1.559733 seconds
0.000000 3.000000 6.000000 9.000000
```

## Compilation en demandant à gcc d'optimiser le code

Comme on le voit, convertir un code avec les jeux d'instructions SIMD manuellement est relativement laborieux et non portable.

Heureusement, le compilateur `gcc` peut vectoriser le code à l'aide d'instructions SIMD, quand elles sont disponibles, et ceci de manière automatique en utilisant les options `-Ofast -march=native`.  

L'option `-fopt-info-vec` (`--ftree-vectorizer-verbose=n` dans les anciennes versions du compilateur) permet d'obtenir des informations sur les optimisations vectorielles réalisées.

La [documentation de `gcc` possède un chapitre entier sur la vectorisation](https://gcc.gnu.org/projects/tree-ssa/vectorization.html) qui vous aidera à trouver les options pour votre processeur.

Par contre il ne saura pas toujours trouver les meilleurs optimisations ni même utiliser ces instructions si le code devient trop complexe.



In [7]:
!gcc compute_simple.c -Ofast -march=native -fopt-info-vec -o compute_simple_gcc.bin

compute_simple.c:14:5: note: loop vectorized
compute_simple.c:14:5: note: loop versioned for vectorization because of possible aliasing
compute_simple.c:14:5: note: loop vectorized


In [8]:
%%time
!./compute_simple_gcc.bin 1000000000 

Using a table of 1000000000 floats
Execution time : 1.257858 seconds
0.000000 3.000000 6.000000 9.000000
CPU times: user 58.1 ms, sys: 22.3 ms, total: 80.4 ms
Wall time: 4.3 s


Nous passons maintenant de - *grosso modo* - 2 secondes à 1 seconde. C'est même bien plus efficace.  Il aura probablement utilisé les instructions `__m256` et d'autres optimisations.

Nous arrêterons ici l'exploration des concepts avec le langage C.  
Pour creuser ce sujet il conviendrait de pratiquer plus en détails ce langage, ce qui n'est pas l'objet de ce cours.

Par contre vous trouverez de nombreuses ressources et exemples sur Internet expliquant comment utiliser ces instructions.

Voici quelques liens pour vous aider à approfondir le sujet:

* Le cours [Introduction à la programmation parallèle](http://kayaogz.github.io/teaching/app4-programmation-parallele-2018/cours/intro.pdf) de M. Oguz Kaya de l'école [Polytech Paris-Sud](http://www.polytech.u-psud.fr/).
* L'article [Programmation parallèle - SIMD et OpenMP](https://www.supinfo.com/articles/single/2115-programmation-parallele-simd-openmp) de M. Xiaoyan CHEN de l'université [SupInfo](https://www.supinfo.com).
* Les [exemples du site GitHub de M. Adrien Guinet](https://github.com/aguinet/misc-examples/tree/master/histogram) dont je me suis inpiré pour créer cet exemple très simplifié.
* Le code source de la librairie de traitement d'images [Simd Library](https://github.com/ermig1979/Simd)

### SIMD avec Python

Python n'est pas en reste et possède quelques librairies qui permettent de faire de la compilation à la volée de vos fonctions en exploitant ces instructions SIMD. 

La librairie [numba](http://numba.pydata.org) est peut-être la plus efficace pour cela - *c'est surtout la seule que je pratique*.  Mais il en existe plusieurs autres dont:

* [Pypy](https://pypy.org/) un fork de l'interpréteur CPython (Celui de python.org). Il réalise automatiquement la compilation *Just In Time*.  
    C'est un projet très actif, [il est compatible avec la majorité des librairies](http://packages.pypy.org/), mais malheureusement pas avec toutes, notamment les scientifiques.
* [Hope](https://github.com/jakeret/hope) est une librairie *JIT* optimisée pour le calcul en astrophysique  
  Elle n'a pas été mise à jour depuis 2016.
* [Pyston](https://blog.pyston.org/) est une autre implémentation de Python intégrant un *JIT* et basé sur [LLVM](https://fr.wikipedia.org/wiki/LLVM).  
Le projet annonce être 96% plus rapide que CPython.  
  Il n'a pas été mis à jour depuis 2017.
* [Nuikta](https://nuitka.net/pages/overview.html) est un compilateur Python écrit en Python !   
**Il génère un exécutable en langage C lié avec libpython!**  
Le projet annonce un gain moyen de 300% et est activement développé !
* [Parakeet](https://github.com/iskandr/parakeet) est un autre *JIT* qui n'est plus maintenu depuis 2015.
* [Pythran](https://pythran.readthedocs.io/en/latest/) est une sorte de compilateur Python qui convertit un module Python en un module natif (librairie c).  
Il sait tirer parti des instructions vectorielles et architectures multicoeurs. C'est un projet de plus en plus actif.

*Numba* permet d'ajouter devant vos fonctions des décorateurs qui vont compiler leur code au moment de l'exécution. 

A la première exécution le code est compilé, ce qui augmente d'autant le temps d'exécution, puis la version compilée est exécutée. Le résultat reste normalement bien plus rapide que le code Python d'origine.  Puis aux appels suivants, le code compilé précédemment est directement repris du cache de *Numba*. Et là, cela va encore plus vite !


Cette technique porte le nom de **Just In Time Compilation**.  



Un petit exemple valant plus qu'un grand discours, reprenons notre addition précédente en pur Python.

La librairie *Numba* n'est pas très à l'aise avec les collections de type *mutable* comme les listes et ensembles (set).  
Aussi, [leur usage est aujourd'hui déprécié](http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types).

L'exemple ci-dessous illustre ce cas de figure.

In [11]:
from numba import jit

@jit
def compute(r, a, b, n):
    
    for ind in range(n):
        r.append(a[ind] + b[ind])

r = []
compute(r, [1,2,3], [4,5,6], 3)

print(r)

[5, 7, 9]


<ipython-input-11-81d1500ec0b8>:3: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "compute" failed type inference due to: non-precise type pyobject
[1] During: typing of argument at <ipython-input-11-81d1500ec0b8> (6)

File "<ipython-input-11-81d1500ec0b8>", line 6:
def compute(r, a, b, n):
    <source elided>
    
    for ind in range(n):
    ^

  @jit
<ipython-input-11-81d1500ec0b8>:3: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "compute" failed type inference due to: cannot determine Numba type of <class 'numba.dispatcher.LiftedLoop'>

File "<ipython-input-11-81d1500ec0b8>", line 6:
def compute(r, a, b, n):
    <source elided>
    
    for ind in range(n):
    ^

  @jit
/home/aliquis/anaconda/lib/python3.7/site-packages/numba/compiler.py:725: NumbaWarning: Function "compute" was compiled in object mode without forceobj=True, but has lifted loops.

File "<ipython-input-11-8

Elle est par contre très efficace avec les tableaux [numpy](https://numpy.org/devdocs/user/quickstart.html).

In [12]:
import numpy as np

def compute(r, a, b, n):
    """For people knewing Numpy this is a very bad use case, I know, this is for training purpose"""
    for ind in range(n):
        r[ind] = a[ind] + b[ind]
        
n = 10**7
data1 = np.arange(n)
data2 = np.arange(n) * 2
result = np.zeros((n,))

%timeit -n 5 -r 2 compute(result, data1, data2, n)

print(result[:4])

2.13 s ± 39 ms per loop (mean ± std. dev. of 2 runs, 5 loops each)
[0. 3. 6. 9.]


Cela n'est pas du tout efficace. 2 secondes contre 1 seconde pour l'équivalent C ayant 100 fois plus d'éléments !

Evidemment cette manière d'utiliser les tableaux numpy n'est pas du tout la bonne.  
Voici la bonne écriture:

In [13]:
%timeit -n 5 -r 2 result = data1 + data2  # numpy + operator automatically adds elements at same position in each table

28.3 ms ± 327 µs per loop (mean ± std. dev. of 2 runs, 5 loops each)


Mais reprenons la fonction `compute` et compilons-la avec *Numba*.

In [14]:
import numpy as np
from numba import jit

@jit(nopython=True)
def compute(r, a, b, n):
    """For people knewing Numpy this is a very bad use case I know, this is for training purpose"""
    for ind in range(n):
        r[ind] = a[ind] + b[ind]
        
n = 10**7
data1 = np.arange(n)
data2 = np.arange(n) * 2
result = np.zeros((n,))

%timeit -n 5 -r 2 compute(result, data1, data2, n)

20.3 ms ± 8.01 ms per loop (mean ± std. dev. of 2 runs, 5 loops each)


20 millisecondes contre 2 secondes dans la version précédente. Cela représente un gain de x100 en ayant juste ajouté le décorateur `@jit(nopython=True)` devant le nom de la fonction.

* `@jit` remplace la fonction d'origine par une version compilée par numba  
  La fonction Python initiale reste accessible via l'attribut `compute.py_func`
* [nopython=True](http://numba.pydata.org/numba-doc/latest/user/5minguide.html#what-is-nopython-mode) indique à numba de générer une version qui n'utilise pas l'interpréteur Python

In [15]:
%timeit -n 2 -r 2 compute.py_func(result, data1, data2, n)  # Orginal function

2.06 s ± 13.7 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)


Essayons maintenant avec un tableau de réels simple précision d'un milliard d'éléments.  
**ATTENTION** il vous faudra plus de **12Go de RAM** pour que cela fonctionne sans utiliser le swap.  
Pensez à changer la valeur de `n` si cela ne vous convient pas.  
Nous spécifions le type `np.float32` qui correspond au type `float` du langage **C**.

In [16]:
n = 10**9
data1 = np.arange(n, dtype=np.float32)
data2 = np.arange(n, dtype=np.float32) * 2
result = np.zeros((n,), dtype=np.float32)

%timeit -n 5 -r 2 compute(result, data1, data2, n)
result[:3]

660 ms ± 66.2 ms per loop (mean ± std. dev. of 2 runs, 5 loops each)


array([0., 3., 6.], dtype=float32)

Et c'est encore plus rapide qu'en laissant `gcc` optimiser la version en langage C !  
C'est plutôt épatant !

Les fonctions compilées avec *Numba* possèdent de nombreux attributs.  
`signatures` permet de consulter les différentes versions de la fonction qui ont été créées par la librairie.

Ici *Numba* aura créé 2 versions: une pour les réels 64 bits et une autre pour les réels 32 bits.

In [17]:
compute.signatures

[(array(float64, 1d, C), array(int64, 1d, C), array(int64, 1d, C), int64),
 (array(float32, 1d, C), array(float32, 1d, C), array(float32, 1d, C), int64)]

La fonction `inspect_asm` permet de regarder le code assembleur généré.  
Elle prend en paramètre une signature de fonction.

In [18]:
text = compute.inspect_asm(signature=compute.signatures[1])
lines = text.split("\n")
for l in lines[:20]:
    print(l)

	.text
	.file	"<string>"
	.globl	_ZN8__main__11compute$248E5ArrayIfLi1E1C7mutable7alignedE5ArrayIfLi1E1C7mutable7alignedE5ArrayIfLi1E1C7mutable7alignedEx
	.p2align	4, 0x90
	.type	_ZN8__main__11compute$248E5ArrayIfLi1E1C7mutable7alignedE5ArrayIfLi1E1C7mutable7alignedE5ArrayIfLi1E1C7mutable7alignedEx,@function
_ZN8__main__11compute$248E5ArrayIfLi1E1C7mutable7alignedE5ArrayIfLi1E1C7mutable7alignedE5ArrayIfLi1E1C7mutable7alignedEx:
	pushq	%rbx
	movq	152(%rsp), %rcx
	testq	%rcx, %rcx
	jle	.LBB0_15
	movq	128(%rsp), %r8
	movq	72(%rsp), %r9
	movq	16(%rsp), %r11
	cmpq	$32, %rcx
	jae	.LBB0_3
	xorl	%eax, %eax
.LBB0_13:
	shlq	$2, %rax
	.p2align	4, 0x90
.LBB0_14:


Les instructions assembleur générées utilisant les registres SIMD ont la forme suivante:

* SSE: `xmm<Number>`
* AVX/AVX2: `ymm<Number>`
* AVX-512: `zmm<Number>`


Nous pouvons ainsi vérifier que le code généré est bien vectorisé !

In [19]:
for l in lines:
    if 'mm' in l:
        print(l)

	vmovss	(%r9,%rax), %xmm0
	vaddss	(%r8,%rax), %xmm0, %xmm0
	vmovss	%xmm0, (%r11,%rax)
	vmovups	(%r9,%rdx,4), %ymm0
	vmovups	32(%r9,%rdx,4), %ymm1
	vmovups	64(%r9,%rdx,4), %ymm2
	vmovups	96(%r9,%rdx,4), %ymm3
	vaddps	(%r8,%rdx,4), %ymm0, %ymm0
	vaddps	32(%r8,%rdx,4), %ymm1, %ymm1
	vaddps	64(%r8,%rdx,4), %ymm2, %ymm2
	vaddps	96(%r8,%rdx,4), %ymm3, %ymm3
	vmovups	%ymm0, (%r11,%rdx,4)
	vmovups	%ymm1, 32(%r11,%rdx,4)
	vmovups	%ymm2, 64(%r11,%rdx,4)
	vmovups	%ymm3, 96(%r11,%rdx,4)
	vmovups	128(%r9,%rdx,4), %ymm0
	vmovups	160(%r9,%rdx,4), %ymm1
	vmovups	192(%r9,%rdx,4), %ymm2
	vmovups	224(%r9,%rdx,4), %ymm3
	vaddps	128(%r8,%rdx,4), %ymm0, %ymm0
	vaddps	160(%r8,%rdx,4), %ymm1, %ymm1
	vaddps	192(%r8,%rdx,4), %ymm2, %ymm2
	vaddps	224(%r8,%rdx,4), %ymm3, %ymm3
	vmovups	%ymm0, 128(%r11,%rdx,4)
	vmovups	%ymm1, 160(%r11,%rdx,4)
	vmovups	%ymm2, 192(%r11,%rdx,4)
	vmovups	%ymm3, 224(%r11,%rdx,4)
	vmovups	(%r9,%rdx,4), %ymm0
	vmovups	32(%r9,%rdx,4), %ymm1
	vmovups	64(%r9,%rdx,4), %ymm2
	vmovups	96(%r9,%

La méthode `inspect_types` est une autre manière d'observer le code généré.  
Dans la sortie affichée il est possible de cliquer sur les lignes affichées pour observer les instructions associées.  

In [22]:
compute.inspect_types(pretty=True)  # click on displayed lines

label 0
"5: def compute(r, a, b, n):"
"6: """"""For people knewing Numpy this is a very bad use case I know, this is for training purpose"""""""
"r = arg(0, name=r) :: array(float64, 1d, C)"
"a = arg(1, name=a) :: array(int64, 1d, C)"
"b = arg(2, name=b) :: array(int64, 1d, C)"
"n = arg(3, name=n) :: int64"
jump 2
label 2
$2.1 = global(range: <class 'range'>) :: Function(<class 'range'>)
"$2.3 = call $2.1(n, func=$2.1, args=[Var(n, <ipython-input-14-f1575a85efed> (7))], kws=(), vararg=None) :: (int64,) -> range_state_int64"


Par contre compiler en utilisant l'opérateur d'addition de *Numpy* n'apporte rien, bien au contraire.  
L'opérateur d'addition de numpy est lui-même implémenté en C, aussi la librairie ne pourra rien faire de mieux.

In [21]:
import numpy as np
from numba import jit

@jit(nopython=True)
def compute(r, a, b):
    """For people knewing Numpy this is a very bad use case I know, this is for training purpose"""
    r = a + b
        
n = 10**7
data1 = np.arange(n)
data2 = np.arange(n) * 2
result = np.zeros((n,))

%timeit -n 5 -r 2 compute(result, data1, data2)
%timeit -n 5 -r 2 compute.py_func(result, data1, data2)

40.3 ms ± 12.2 ms per loop (mean ± std. dev. of 2 runs, 5 loops each)
28.3 ms ± 132 µs per loop (mean ± std. dev. of 2 runs, 5 loops each)


La documentation de *Numba* fournit énormément d'exemples complémentaires pour exploiter au mieux cette librairie qui peut aussi compiler pour GPU ou paralléliser les calculs sur les différents coeurs de votre CPU

Pour approfondir les possibilités SIMD de *Numba*, voici le lien vers le notebook du sous-projet d'exemples présentant cette fonctionnalité: 

https://github.com/numba/numba-examples/blob/master/notebooks/simd.ipynb

Bonne compilation !